In [0]:

import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from google.colab import files

# Upload dataset

- sign up for a Kaggle account at https://www.kaggle.com. 
-  go to the 'Account' tab of your user profile https://www.kaggle.com/<username>/accoun
- select 'Create API Token'.
- This will trigger the download of kaggle.json, a file containing your API credentials
  

In [0]:
!pip install -U -q kaggle
!mkdir  /root/.kaggle

files.upload()
!cp kaggle.json /root/.kaggle
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia


# Prepare dataset

In [0]:
!apt install pv
!unzip -o /content/chest-xray-pneumonia.zip  | pv -l >/dev/null
os.remove('chest-xray-pneumonia.zip')
!unzip -o /content/chest_xray.zip  | pv -l >/dev/null
os.remove('chest_xray.zip')

# Make model

In [10]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [0]:
model=Model(inputs=base_model.input,outputs=preds)

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
    

# Train model

In [13]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/chest_xray/train/',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=32,
                                                  class_mode='categorical', shuffle=True)


model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

Found 5216 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
163/163 [==============================] - 78s 478ms/step - loss: 0.1360 - acc: 0.9546
Epoch 2/5
163/163 [==============================] - 76s 468ms/step - loss: 0.0435 - acc: 0.9833
Epoch 3/5
163/163 [==============================] - 75s 461ms/step - loss: 0.0360 - acc: 0.9875
Epoch 4/5
163/163 [==============================] - 75s 460ms/step - loss: 0.0454 - acc: 0.9872
Epoch 5/5
163/163 [==============================] - 75s 459ms/step - loss: 0.0339 - acc: 0.9898


#Save the weights

In [14]:
model.save('chest-xray-pneumonia.h5')    
!zip -r model.zip 'chest-xray-pneumonia.h5'

  adding: chest-xray-pneumonia.h5 (deflated 13%)


# Implement

In [0]:
from keras.models import load_model
new_model = load_model("/content/chest-xray-pneumonia.h5")

def get_rez(pic):
  img = image.load_img(pic, target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  p_good,p_ill = np.around(new_model.predict(x), decimals=2)[0]
  return{'p_good':p_good,'p_ill':p_ill}

In [24]:
ill_path = "/content/chest_xray/train/PNEUMONIA/" 
good_path = "/content/chest_xray/train/NORMAL/" 

ill_pic = ill_path + os.listdir(ill_path)[0]
good_pic = good_path + os.listdir(good_path)[5]

print(get_rez(ill_pic))
print(get_rez(good_pic))



{'p_good': 0.0, 'p_ill': 1.0}
{'p_good': 0.06, 'p_ill': 0.94}
